# Build a Chatbot

LangChain v0.3 릴리스부터는 LangChain 사용자들이 memory를 새로운 LangChain 애플리케이션에 통합하기 위해 LangGraph 지속성(LangGraph Persistence)을 사용하는 것이 권장됩니다.

이미 RunnableWithMessageHistory 또는 BaseChatMessageHistory에 의존하고 있는 코드라면 아무 변경도 필요하지 않습니다. 이 기능은 간단한 채팅 애플리케이션에서 계속 사용할 수 있으며, RunnableWithMessageHistory를 사용하는 코드는 기대한 대로 계속 작동할 것입니다.

### 개요 (Overview)
LLM을 활용한 챗봇 설계 및 구현 예제를 살펴보겠습니다.
이 챗봇은 대화를 나누고 이전 상호작용을 기억할 수 있습니다.

이 챗봇은 언어 모델만 사용하여 대화를 진행합니다.
다음과 같은 관련 개념도 확인할 수 있습니다:

- Conversational RAG: 외부 데이터 소스를 활용한 챗봇
- 에이전트(Agents): 액션을 수행할 수 있는 챗봇


In [ ]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv \
# langgraph

In [ ]:
# LangChain 추적(Tracing) 설정 활성화

먼저 모델을 직접 사용해 봅니다. `ChatModel`은 LangChain의 **"Runnable"** 인스턴스이며, 이는 표준화된 인터페이스를 통해 상호작용할 수 있음을 의미합니다.  

모델을 간단하게 호출하려면 `.invoke` 메서드에 **메시지 목록**을 전달하면 됩니다.

모델 자체는 **상태(state)**라는 개념을 가지고 있지 않습니다. 예를 들어, 후속 질문을 하면:

이제 좋은 응답을 받는 것을 확인할 수 있습니다!  

이것이 챗봇이 **대화형 상호작용**을 할 수 있는 기본 아이디어입니다.  
그렇다면, 이를 가장 효과적으로 구현하는 방법은 무엇일까요?

## **메시지 지속성 (Message Persistence)**  

**LangGraph**는 **내장된 지속성 계층(persistence layer)**을 구현하여 **여러 번의 대화(turns)**를 지원하는 챗 애플리케이션에 이상적입니다.  

챗 모델을 간단한 LangGraph 애플리케이션으로 감싸면 **메시지 기록을 자동으로 저장(persist)**할 수 있어, **다중 턴(multi-turn)** 애플리케이션 개발이 훨씬 더 간편해집니다.  

LangGraph에는 **간단한 인메모리 체크포인터(in-memory checkpointer)**가 포함되어 있으며, 아래 예제에서 이를 사용합니다.  

In [ ]:
# 새로운 그래프 정의
# 메시지 상태(MessagesState)를 상태 스키마(state_schema)로 사용하여 워크플로우를 정의
# 모델을 호출하는 함수 정의
def call_model(state: MessagesState):
# 그래프 노드 및 엣지 설정
# START 지점에서 "model" 노드로 이동하도록 엣지를 추가합니다.
# "model" 노드에 call_model 함수를 연결합니다.
# 메모리 추가 (메시지 기록 저장)
# 워크플로우를 메모리 체크포인트와 함께 컴파일합니다.

이제 매번 `runnable` 객체에 전달할 **`config`**를 생성해야 합니다.  

이 **`config`**에는 입력에 직접 포함되지는 않지만 유용한 정보가 포함됩니다.  이번 경우, **`thread_id`**를 포함할 것 입니다.  

다음과 같은 형태여야 합니다:

이렇게 하면 하나의 애플리케이션에서 **여러 대화 스레드(thread)**를 지원할 수 있습니다.  

여러 사용자가 애플리케이션을 동시에 사용할 때 흔히 필요한 기능입니다.  

이제 애플리케이션을 호출할 수 있습니다:

In [ ]:
# 입력 메시지를 리스트 형태로 정의
# 이는 대화의 첫 번째 메시지가 됩니다.
# 'app.invoke'를 사용하여 애플리케이션을 호출
# 첫 번째 매개변수: 입력 메시지
# 두 번째 매개변수: 추가적인 설정(config) - 예: thread_id 등
# 상태(state)에 있는 모든 메시지 중 마지막 메시지 출력

이제 우리의 챗봇은 우리에 대한 정보를 기억합니다.  

`config`에서 다른 **`thread_id`**를 참조하도록 변경하면, 챗봇이 **새로운 대화**를 시작하는 것을 확인할 수 있습니다.

그러나 우리는 항상 **원래의 대화로 돌아갈 수 있습니다** (데이터베이스에 대화를 **저장(persisting)**하고 있기 때문입니다).

이렇게 하면 챗봇이 여러 사용자와 동시에 대화를 할 수 있습니다.

### 비동기(async) 지원
비동기 지원을 위해 call_model 노드를 **비동기 함수(async function)**로 업데이트하고, 애플리케이션을 호출할 때 .ainvoke를 사용하세요:

```python
# 노드의 비동기 함수:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

# 이전과 동일하게 그래프 정의:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# 비동기 호출:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

```

지금까지 우리는 모델 주위에 간단한 **지속성 계층(persistence layer)**을 추가한 것에 불과합니다. 이제 **프롬프트 템플릿(prompt template)**을 추가하여 챗봇을 더 복잡하고 개인화된 형태로 만들어 봅시다.

---

## ** 프롬프트 템플릿 (Prompt Templates)**  

**프롬프트 템플릿**은 **원시 사용자 입력(raw user input)**을 LLM이 처리할 수 있는 형식으로 변환하는 데 도움을 줍니다.   

1. 먼저, **시스템 메시지(system message)**를 추가하여 **사용자 정의 지침(custom instructions)**을 포함시킵니다. (여전히 메시지를 입력으로 사용)  
2. 다음으로, 메시지 외에 **더 많은 입력 정보**를 추가합니다.  

---

### ** 시스템 메시지(System Message) 추가하기**  

시스템 메시지를 추가하기 위해 **`ChatPromptTemplate`**을 생성합니다.  
여기서는 모든 메시지를 전달하기 위해 **`MessagesPlaceholder`**를 사용할 것입니다.  

이렇게 하면 LLM에 전달되는 입력이 더 구조화되고, 챗봇의 동작을 더 정교하게 제어할 수 있습니다.  

In [ ]:
# LLM이 사용자 입력을 더 잘 처리할 수 있도록 프롬프트 템플릿을 설정합니다.
        # LLM의 동작 방식을 정의하는 지침
        # Messages Placeholder - 이전 대화 메시지들을 전달합

이제 이 템플릿을 통합하여 애플리케이션을 업데이트할 수 있습니다.

In [ ]:
# MessagesState를 상태 스키마로 사용하여 워크플로우를 정의합니다.
# 모델 호출 함수 정의
def call_model(state: MessagesState):
    # 프롬프트 템플릿을 사용하여 상태(state)로부터 프롬프트를 생성합니다.
    # 생성된 프롬프트를 모델에 전달하고 응답을 받습니다.
    # 응답을 딕셔너리 형태로 반환합니다.
# START에서 "model" 노드로 이동하도록 엣지 정의
# "model" 노드에 call_model 함수를 연결
# MemorySaver를 사용하여 상태 및 메시지 기록을 저장합니다.
# 그래프를 메모리 체크포인트와 함께 컴파일합니다.

같은 방식으로 응용 프로그램을 호출합니다:

In [ ]:
# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
# 메시지 목록에 사용자 메시지 추가
# 애플리케이션 호출
# 메시지 상태(State), 설정(config) 전달

프롬프트를 조금 더 복잡하게 만들어 봅니다. 프롬프트 템플릿이 이제 다음과 같다고 가정해 봅니다.

우리는 프롬프트에 새로운 **`language`** 입력을 추가했습니다.  

이제 우리의 애플리케이션은 두 개의 매개변수를 가지게 되었습니다:  
- **`messages`**: 입력 메시지  
- **`language`**: 언어 설정  

이 변경사항을 반영하기 위해 애플리케이션의 **상태(state)**를 업데이트해야 합니다.

In [ ]:
# 새로운 상태 스키마에 'messages'와 'language'를 추가합니다.
class State(TypedDict):
    # 메시지 시퀀스를 정의하고 add_messages 도구를 사용하여 관리합니다.
    # 언어 설정을 위한 문자열 필드
# 상태 스키마(State Schema)로 'State'를 사용하여 그래프를 정의합니다.
# 모델 호출 함수 정의
def call_model(state: State):
    # 상태에서 프롬프트 생성.
    # 생성된 프롬프트를 모델에 전달하고 응답을 받습니다.
# MemorySaver를 사용하여 상태 및 메시지 기록을 저장합니다.
# 그래프를 메모리 체크포인트와 함께 컴파일합니다.

In [ ]:
#  'thread_id'를 사용하여 특정 대화 스레드를 식별
# HumanMessage 객체를 메시지 목록에 추가
# 애플리케이션 호출

전체 상태가 유지되므로 변경이 필요하지 않으면 `language`와 같은 매개변수를 생략할 수 있습니다.

내부에서 무슨 일이 일어나고 있는지 이해하는 데 도움이 되도록 [LangSmith 추적](https://smith.langchain.com/o/351c6cd9-1396-5c74-9478-1ee6a22a6433/projects/p/acec9d4d-4978-4597-adff-789cd42e200f?timeModel=%7B%22duration%22%3A%227d%22%7D)을 확인하세요.

### 대화 이력 관리  

챗봇을 구축할 때 중요한 개념 중 하나는 **대화 이력 관리**입니다. 이력이 관리되지 않으면 메시지 목록이 무한정으로 증가해 LLM의 **컨텍스트 윈도우를 초과할 가능성**이 있습니다. 따라서 전달하는 메시지의 크기를 제한하는 단계를 추가하는 것이 중요합니다.  

**중요한 점은, 이 단계는 프롬프트 템플릿 이전에, 그러나 메시지 이력을 불러온 이후에 수행해야 한다는 것입니다.**  

이를 위해 프롬프트 앞단에 `messages` 키를 적절히 수정하는 단계를 추가한 후, 이 새 체인을 **Message History 클래스**로 감싸면 됩니다.  

**LangChain**에는 메시지 목록을 관리하기 위한 몇 가지 내장 도구가 제공됩니다. 여기서는 trim_messages 헬퍼를 사용하여 모델에 전달하는 메시지 수를 줄일 것입니다.  

`trim_messages`를 사용하면 보유할 토큰 수, 시스템 메시지를 항상 유지할지 여부, 부분 메시지를 허용할지 여부와 같은 매개변수를 지정할 수 있습니다.

In [ ]:
# 메시지 이력을 다듬기 위한 트리머(trimmer) 설정
# 메시지 이력 예제
# 트리머를 사용하여 메시지 이력을 다듬기
# 설정된 조건에 따라 오래된 메시지 또는 불필요한 메시지가 제거됨
# 다듬어진 메시지 출력 (디버깅 또는 확인용)

이를 체인에서 사용하려면 프롬프트에 `messages` 입력을 전달하기 전에 트리머를 실행하기만 하면 됩니다.

In [ ]:
# 상태(State) 그래프 워크플로우 정의
# 모델 호출 함수 정의
def call_model(state: State):
    # 1. # 메시지 이력을 트리밍하여 토큰 수 제한
    # 2. 프롬프트 템플릿 적용
    # 3. LLM 호출
    # 4. 응답 메시지 반환
# 워크플로우 노드 및 엣지 추가
# 상태를 저장 및 복원할 메모리 체크포인터 생성
# 워크플로우 컴파일

이제 우리가 모델에게 우리의 이름을 물어보려고 하면, 우리가 채팅 기록의 해당 부분을 잘라냈기 때문에 모델이 우리의 이름을 알 수 없을 것입니다.

In [ ]:
# highlight-next-line

하지만 최근 몇 개의 메시지에 있는 정보에 대해 묻는다면, 그것은 다음을 기억합니다:

LangSmith를 살펴보면 [LangSmith 추적](https://smith.langchain.com/o/351c6cd9-1396-5c74-9478-1ee6a22a6433/projects/p/acec9d4d-4978-4597-adff-789cd42e200f?timeModel=%7B%22duration%22%3A%227d%22%7D)에서 정확히 무슨 일이 일어나고 있는지 확인할 수 있습니다.

## **스트리밍 (Streaming)**  

이제 우리는 작동하는 챗봇을 만들었습니다. 그러나 챗봇 애플리케이션의 사용자 경험(UX)에서 **매우 중요한 요소** 중 하나는 **스트리밍(Streaming)**입니다.  

LLM은 때때로 응답을 생성하는 데 시간이 걸릴 수 있습니다. 따라서 사용자 경험을 개선하기 위해 대부분의 애플리케이션은 **생성되는 각 토큰을 실시간으로 스트리밍**하여 사용자에게 보여줍니다. 이렇게 하면 사용자는 응답의 진행 상황을 실시간으로 확인할 수 있습니다.  

사실, 이 작업은 매우 간단합니다!  

기본적으로 LangGraph 애플리케이션에서 `.stream`은 애플리케이션 단계(예: 모델 응답 단계)를 스트리밍합니다. 그러나 `stream_mode="messages"`를 설정하면 **출력 토큰을 실시간으로 스트리밍**할 수 있습니다.  

In [ ]:
# 스트리밍 설정을 위한 구성(config) 정의
# 사용자 입력 메시지 및 언어 설정
# 입력 메시지를 HumanMessage 객체로 변환
# LangChain 애플리케이션 스트리밍 시작
    # AIMessage 객체만 필터링하여 출력

## **다음 단계**

LangChain을 사용해 챗봇을 만드는 기본 개념을 이해했다면, 다음과 같은 **고급 튜토리얼**에 관심이 있을 수 있습니다:

- **[대화형 RAG](https://python.langchain.com/docs/tutorials/qa_chat_history):** 외부 데이터 소스를 활용하여 챗봇 경험을 확장하기  
- **[에이전트](https://python.langchain.com/docs/tutorials/agents):** 특정 작업을 수행할 수 있는 챗봇 구축하기  

**더 깊이 dive-in**하고 싶다면, 다음 항목들을 확인해보세요:

- **[스트리밍](https://python.langchain.com/docs/how_to/streaming):** 챗 애플리케이션에서 *매우 중요한* 스트리밍 관리  
- **[메시지 이력 추가 방법](https://python.langchain.com/docs/how_to/message_history):** 메시지 이력 관리의 모든 것 자세히 살펴보기  
- **[대규모 메시지 이력 관리 방법](https://python.langchain.com/docs/how_to/trim_messages/):** 방대한 대화 이력을 효율적으로 관리하는 기술  
- **[LangGraph 공식 문서](https://langchain-ai.github.io/langgraph/):** LangGraph를 활용한 빌딩에 대한 심층적인 가이드  

이 자료들을 통해 더욱 정교하고 강력한 챗봇을 구축할 수 있습니다! 🚀